In [1]:
import sys
sys.path.append('../..')
%load_ext autoreload
%autoreload 2

In [2]:
import spatialpy
import numpy
import matplotlib.pyplot as plt


In [3]:
class All(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        mask = x[0] < numpy.Inf
        return mask

class Top(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        mask = (x[1] <= 1.0) & (x[1] > 0.95)
        return mask

In [4]:
class Simple_Diffusion(spatialpy.Model):
    """ Initial condition is a delta function at and off-center point. 
        The solution should be a Gaussian, and periodic boundary conditions
        should allow it to wrap. """

    def __init__(self, model_name="simple_diffusion"):
        spatialpy.Model.__init__(self,model_name)
        
        # System constants
        D = 0.01

        # Define Species
        A = spatialpy.Species(name="A",diffusion_constant=D)
        self.add_species([A])

        # A unit square
        self.mesh = spatialpy.Mesh.generate_unit_square_mesh(40,40, periodic=True)

        # Define Subdomains
        self.add_subdomain(All(), 1) # total subdomain, type 1
        self.add_subdomain(Top(), 2) # top subdomains, type 2


        # Place the A molecules in the voxel nearest the center of the square
        self.add_initial_condition(
            spatialpy.PlaceInitialCondition(A,100000,[0,0,0])
        )

        self.timespan(numpy.linspace(0,5,1000))

In [5]:
model = Simple_Diffusion()

In [6]:
model.listOfSubdomainIDs

[1, 2]

In [7]:
%time result = model.run()

[]
CPU times: user 17 s, sys: 665 ms, total: 17.7 s
Wall time: 27.3 s


In [8]:
result.display('A',-1)

Exception: TODO

In [9]:
def display(result, species, timepoint, opacity=1.0, wireframe=True, width=500):
    data = result.get_species(species,timepoint,concentration=True)
    fun = pyurdme.DolfinFunctionWrapper(result.model.mesh.get_function_space())
    vec = fun.vector()
    nd = data.shape[0]
    if nd == len(vec):
        for i in range(nd):
            vec[i]=data[i]
    else:
        v2d= result.get_v2d()
        for i in range(len(vec)):
            vec[i]=data[i]
    fun.display(opacity=opacity, wireframe=wireframe, width=width)

In [10]:
display(result, 'A', 200)

ValueError: too many values to unpack (expected 4)

In [11]:
model.mesh